<a href="https://colab.research.google.com/github/atilaye/BioFaceNet-Python-Implementation/blob/master/BioFaceNet_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


This code is a Python translation of the MATLAB implementation of BioFaceNet, originally provided by the authors of the paper:
S. Alhagry, R. Bahgat, A. Elsayed, "BioFaceNet: Deep Convolutional Neural Network for Face Photo-Sketch Recognition," arXiv:1908.10578 [cs.CV], 2019.
https://arxiv.org/pdf/1908.10578

Original MATLAB code repository: https://github.com/ssma502/BioFaces

Translated by: Ameya Tilaye
Date: 9/22/2024



CelebA dataset:
@inproceedings{conf/iccv/LiuLWT15,
  added-at = {2018-10-09T00:00:00.000+0200},
  author = {Liu, Ziwei and Luo, Ping and Wang, Xiaogang and Tang, Xiaoou},
  biburl = {https://www.bibsonomy.org/bibtex/250e4959be61db325d2f02c1d8cd7bfbb/dblp},
  booktitle = {ICCV},
  crossref = {conf/iccv/2015},
  ee = {http://doi.ieeecomputersociety.org/10.1109/ICCV.2015.425},
  interhash = {3f735aaa11957e73914bbe2ca9d5e702},
  intrahash = {50e4959be61db325d2f02c1d8cd7bfbb},
  isbn = {978-1-4673-8391-2},
  keywords = {dblp},
  pages = {3730-3738},
  publisher = {IEEE Computer Society},
  timestamp = {2018-10-11T11:43:28.000+0200},
  title = {Deep Learning Face Attributes in the Wild.},
  url = {http://dblp.uni-trier.de/db/conf/iccv/iccv2015.html#LiuLWT15},
  year = 2015
}


In [ ]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets

In [ ]:
import scipy.io as spio
from scipy.io import loadmat
import torch
import numpy as np
from google.colab import drive
drive.mount('/content/drive/MyDrive/Visual Machines Group/')  #CelebA is downloaded from Google Drive


ValueError: Mountpoint must not contain a space.

In [ ]:
#import celebmat here
import torchvision


from torchvision.datasets import ImageFolder
from torchvision import transforms

from torch.utils.data import Dataset, DataLoader
import pandas as pd
from skimage import io
class CelebDataset(Dataset):                #importing CelebA dataset
      def __init__(self,data_dir,partition_file_path,split,transform):
          self.partition_file = pd.read_csv(partition_file_path)
          self.data_dir = data_dir
          self.split = split
          self.transform = transform
      def __len__(self):
          self.partition_file_sub = self.partition_file[self.partition_file["partition"].isin(self.split)]
          return len(self.partition_file_sub)
      def __getitem__(self,idx):
          img_name = os.path.join(self.data_dir,
                                  self.partition_file_sub.iloc[idx, 0])
          image = io.imread(img_name)
          if self.transform:
              image = self.transform(image)
          return image

batch_size = 50000

transform = transforms.Compose(       #batch transformation
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


#download CelebA dataset

data_loader = torch.utils.data.DataLoader(celeba_data,batch_size=4,shuffle=True,num_workers = *args.nThreads) #change batch size once you get this to work

'''
'''
torchvision.datasets

imagenet_data = torchvision.datasets.ImageNet('path/to/imagenet_root/')
data_loader = torch.utils.data.DataLoader(imagenet_data,
                                          batch_size=4,
                                          shuffle=True,
                                          num_workers=args.nThreads)

celeba:

torchvision.datasets.CelebA(root: str, split: str = 'train', target_type: Union[List[str], str] = 'attr', transform: Optional[Callable] = None,target_transform: Optional[Callable] = None, download: bool = False)
'''

In [ ]:
Newskincolour = spio.loadmat('/content/drive/MyDrive/Visual Machines Group/Newskincolour.mat') #'imdb/celebaimdb.mat'

In [ ]:
Tmatrix = spio.loadmat('/content/drive/MyDrive/Visual Machines Group/Tmatrix.mat')

In [ ]:
illF = spio.loadmat('/content/drive/MyDrive/Visual Machines Group/illF.mat')

In [ ]:
illumA = spio.loadmat('/content/drive/MyDrive/Visual Machines Group/illumA.mat')

In [ ]:
illumDmeasured = spio.loadmat('/content/drive/MyDrive/Visual Machines Group/illumDmeasured.mat')

In [ ]:
cmf = spio.loadmat('/content/drive/MyDrive/Visual Machines Group/rgbCMF.mat')


In [ ]:
xyz_space = spio.loadmat('/content/drive/MyDrive/Visual Machines Group/XYZspace (1).mat')

In [ ]:
#CameraSensitivityPCA.m
#PCA Model for Camera Sensitivities
#function [mu,PC,EV]= CameraSensitivityPCA(cmf) returns array
mu, PC, EVpca = CameraSensitivityPCA(cmf)

def diagonalize(A): #Diagonalized matrix D = E^-1 * A * E
  #A = np.sqrt(EV[1:2])
  E = np.linalg.eig(A)
  E_inv = np.linalg.inv(A)
  X = np.dot(E_inv,A)
  D = np.dot(X,E)
  return D

from sklearn.decomposition import PCA
def CameraSensitivityPCA(cmf):
  X = np.zeros((99,28))
  Y = np.zeros((99,28))
  redS = cmf.rgbCMF[1][1] #not sure if I need the cmf.
  greenS = cmf.rgbCMF[1][2]
  blueS = cmf.rgbCMF[1][3]
  for i in range(1,28):
    a = redS[:,i]
    b = np.sum(redS[:,i])
    Y[1:33,i]= np.linalg.lstsq(a.T, b.T)[0].T #np.true_divide(redS[:,i],sum(redS[:,i])) #true right array division
    c = greenS[:,i]
    d = np.sum(greenS[:,i])
    Y[34:66,i]= np.linalg.lstsq(c.T, d.T)[0].T #np.true_divide(greenS[:,i]) #greenS[:,i]./sum(greenS(:,i))
    e = blueS[:,i]
    f = np.sum(blueS[:,i])
    Y[67:99,i]= np.linalg.lstsq(e.T, f.T)[0].T #blueS(:,i)./sum(blueS(:,i))
    break
  pca = PCA(Y.transpose())  #[PC,~,EV,~,explained,mu] = pca(Y'); #principal component coefficients of the matrix transpose of Y
  mu = np.float32(mu.transpose())
  EV = np.float32(EV[1:2])
  A = np.sqrt(EV[1:2])
  PC = np.float32(PC[:,1:2] * diagonalize(A)) #PC = np.float32(PC[:,1:2] * diagonalized sqrt(EV[1:2]))
  return [mu,PC,EV]

In [ ]:
#setup.m
#single to np.float32

import torch

#[mu,PC,EVpca] = CameraSensitivityPCA(cmf);

#predicted vectors' sizes
LightVectorSize = np.float32(15)
wavelength = np.float32(33)
bSize = np.float32(2)

#normalize illuminations
illF = np.float32(illF.reshape(1,1,33,12))


illumDmeasured = (illumDmeasured.transpose())
illumDmeasured = np.reshape(illumDmeasured,(1,1,33,22))

#A (what is A?)
#illumA         = single(illumA./sum(illumA(:)));
illumA =  np.float32(np.linalg.lstsq(illumA.T, np.sum(illumA[:]).T)[0].T)

e = blueS[:, i]  # Extract the i-th column
f = np.sum(blueS[:, i])  # Sum of the i-th column
Y[67:99, i] = blueS[:, i] / np.sum(blueS[:, i])  # Normalized column

#D
illumDNorm = np.float32(np.zeros(1,1,33,22))
for i in range(1,22):
  a = illumDmeasured[1,1,:,i]
  b = np.sum(illumDmeasured[1,1,:,i])
  illumDNorm[1,1,:,i] = np.linalg.lstsq(a.T, b.T)[0].T
  break
# Normalizing illumF
illumFNorm = np.zeros((1, 1, 33, 12), dtype=np.float32)
for i in range(12):  # Python indexing starts at 0, MATLAB uses 1-based indexing
    illumFNorm[0, 0, :, i] = illF[0, 0, :, i] / np.sum(illF[0, 0, :, i])


# Define mu and PC as PyTorch tensors
mu = torch.tensor(mu, dtype=torch.float32, requires_grad=False)  # Create tensor
PC = torch.tensor(PC, dtype=torch.float32, requires_grad=False)  # Create tensor

# Move to GPU
mu = mu.cuda()  # Transfer to GPU
PC = PC.cuda()

# Same for illumA and illumFNorm
illumA = torch.tensor(illumA, dtype=torch.float32, requires_grad=False)
illumA = illumA.cuda()

illumFNorm = torch.tensor(illumFNorm, dtype=torch.float32, requires_grad=False)
illumFNorm = illumFNorm.cuda()


In [ ]:
#findT.m
def findT(Tmatrix,BGrid): #BGrid defined in scalingNet.m
  Tmatrix = spio.loadmat('/content/drive/MyDrive/Visual Machines Group/Tmatrix.mat')
  T_RAW2XYZ = nn.Bilinear(Tmatrix,BGrid)


In [ ]:

#fcn_multipleDecoders.m
def multipleDecoders(x,y,nfilters,doubleconv):
  '''
  %FCN Build an autonn fully convolutional network,image to image tasks, skip connections from each encoder layer to corresponding
% decoder layer
%   Inputs kept the same:
%       nfilters - vector of filter sizes, 1:length-1 are used in the
%       contractive part of the network, the final value is the number of
%       filters applied at the lowest resolution before upsampling begins
%       x - input feature map as autonn layer
%       nclass - number of channels in output feature map
%
%
%   Outputs kept the same:
%       x - image to image prediction
%       y - output of lowest resolution - can be fed into fully connected
%       layers

  '''
#nfilters = [length for length in range (1,length-1)]
nfilters = [32,32,64,96,128,256] #don't know how to read in array in Python
nlayers = np.array(np.size(nfilters)) #nlayers is a vector
for i in range(1, nlayers-1):
    if i == 1:
        #x = nn.Conv1d(x,[3 3 3 nfilters(i)], np.pad()) #figure out padding
        x = nn.Conv1d(i,np.size(nlayers),np.pad([3, 3, 3, nfilters[i]]), stride = 1)
    else:
        #x = nn.Conv1d(x, np.pad([3 3 nfilters(i-1) nfilters(i)]))
        x = nn.Conv1d(i,np.size(nlayers), np.pad([3, 3, nfilters[i-1], nfilters[i]]), stride = 1)
    break
    x = nn.BatchNorm1d(x)
    if doubleconv:#boolean
      x = nn.ReLU(x)
      #x = nn.Conv1d(x,[3 3 nfilters(i) nfilters(i)] , np.pad())
      x = nn.Conv1d(x,np.pad([3, 3, nfilters[i], nfilters[i]]), stride = 1)
      x = nn.BatchNorm1d(x) #batch normalization
      x = nn.ReLU(x)
      #x = nn.Conv1d(x,[3 3 nfilters(i) nfilters(i)] , np.pad())
      x = nn.Conv1d(x,np.pad([3, 3, nfilters[i], nfilters[i]]), stride = 1)
      x = nn.BatchNorm1d(x)
    break
    x_skip[i] = nn.ReLu(x) #x_skip{i} = vl_nnrelu(x);
    x = nn.MaxPool1d(x_skip[i],2,stride = 2) #x = vl_nnpool(x_skip{i}, 2, 'stride', 2) ;
break

x = nn.Conv1d(x,np.pad([3, 3, nfilters[nlayers-1], nfilters[nlayers]]), stride = 1)
x = nn.BatchNorm1d(x) #batch normalization
y = nn.ReLU(x)
if doubleconv:
   x = nn.Conv1d(y,np.pad([3, 3, nfilters[nlayers], nfilters[nlayers]]), stride = 1)
   x = nn.BatchNorm1d(x) #batch normalization
   y = nn.ReLU(x)
   x = nn.Conv1d(y,np.pad([3, 3, nfilters[nlayers], nfilters[nlayers]]), stride = 1)
   x = nn.BatchNorm1d(x) #batch normalization
   y = nn.ReLU(x)
break


#Decoders
#vl_nnupsample = Layer.fromFunction(@autonn_upSample);

for c in range(1, nclass):
  for i in range(nlayers-1,-1,1):
    if i == nlayers-1:
    #m = nn.Upsample(scale_factor=2, mode='nearest')
      x = nn.upSample(y)
    else:
      x = nn.upSample(x)
      break
    #x = cat(3,x,x_skip{i});
      x = nn.Conv1d(x,np.pad([3, 3, nfilters[i]+nfilters[i+1], nfilters[i]]), stride = 1)
      x = nn.BatchNorm1d(x) #batch normalization
      x = nn.ReLU(x)

    if doubleconv:
      x = nn.Conv1d(x,np.pad([3, 3, nfilters[i], nfilters[i]]), stride = 1)
      x = nn.BatchNorm1d(x) #batch normalization
      x = nn.ReLU(x)

      x = nn.Conv1d(x,np.pad([3, 3, nfilters[i], nfilters[i]]), stride = 1)
      x = nn.BatchNorm1d(x) #batch normalization
      x = nn.ReLU(x)
    break
  break

  #Final Predictions

  x = nn.Conv1d(x,np.pad([3, 3, nfilters[1], 1]), stride = 1)
  if c==1:
    z = x
  else:
    z = np.concatenate((z,x), axis = 3) # z = cat(3,z,x);
  break
break

In [ ]:
#CNN.m

def CNN(images,nfilters,nclass,LightVectorSize,bSize):

  """

Inputs:,
    images           : input to CNN
     nfilters         : vector of filter sizes
    nclass           : number of output maps
    lightVectorLSize : 15
     bsize            : 2
Output:
     weightA    : 1 x 1 x 1 x B
     weightD    : 1 x 1 x 1 x B
     CCT        : 1 x 1 x 1 x B
     Fweights   : 1 x 1 x 12 x B
     b          : 2 x nbatch

  """
-------------------------------- CNN -------------------------------------
x,y  = fcn_multipleDecoders(nfilters,images,nclass, True)

import torch
import torch.nn as nn
import torch.nn.functional as F

# Assuming `y` is a tensor of shape (batch_size, channels, height, width)

# FC1 (Convolution + BatchNorm + ReLU)
fc1 = nn.Conv2d(512, 512, kernel_size=4)(y)  # [4, 4, 512, 512]
fc1 = nn.BatchNorm2d(512)(fc1)
fc1 = F.relu(fc1)

# FC2 (Convolution + BatchNorm + ReLU)
fc2 = nn.Conv2d(512, 512, kernel_size=1)(fc1)  # [1, 1, 512, 512]
fc2 = nn.BatchNorm2d(512)(fc2)
fc2 = F.relu(fc2)

# FC3 (Convolution)
dims = LightVectorSize + bSize  # Set this variable beforehand
prediction = nn.Conv2d(512, dims, kernel_size=1)(fc2)
prediction.name = 'prediction'

# Illumination parameters
lightingparameters = prediction[:, :, :LightVectorSize, :]

# Camera parameters
nbatch = prediction.shape[3]  # Assuming batch_size is the 4th dimension
b = prediction[:, :, LightVectorSize:LightVectorSize + bSize, :].view(bSize, nbatch)

# Predicted Maps
fmel = x[:, :, 0, :]
fblood = x[:, :, 1, :]
Shading = x[:, :, 2, :]
specmask = x[:, :, 3, :]



In [ ]:
#ImageFormation.m

def ImageFormation (R_total, Sr,Sg,Sb,e,Specularities,Shading):
  """
Inputs:
    R_total       : H X W X 33 X nbatch
    Shading       : H X W X 1 X nbatch
    Specularities : H X W X 1 X nbatch
    Sr,Sg,Sb      : 1 x 1 x 33 x nbatch
    e             : 1 x 1 x 33 x nbatch
Output:
    rgbim : H x W x 1 x nbatch
"""

import numpy as np

 #--------------------------- Image Formation -------------------------------
spectraRef = R_total * e  # Element-wise multiplication in NumPy

# Calculate the r, g, b channels by summing along axis 3 (channel dimension)
rChannel = np.sum(spectraRef * Sr, axis=2)
gChannel = np.sum(spectraRef * Sg, axis=2)
bChannel = np.sum(spectraRef * Sb, axis=2)

# Stack the r, g, b channels along a new axis (axis 2, equivalent to concatenation along 3rd axis in MATLAB)
diffuseAlbedo = np.stack((rChannel, gChannel, bChannel), axis=2)  # Shape: H x W x 3

# --------------------------- Shaded Diffuse --------------------------------

# Element-wise multiplication to get Shaded Diffuse
ShadedDiffuse = diffuseAlbedo * Shading  # Assuming element-wise multiplication

# --------------------------- Raw Appearance --------------------------------
# Sum the ShadedDiffuse with Specularities
rawAppearance = ShadedDiffuse + Specularities  # Assuming element-wise addition




In [ ]:
#cameraModel.m

def cameraModel(mu,PC,b,wavelength):
"""
Inputs:
  mu         : 1 x 1 x 1 x B
  PC         : 1 x 1 x 1 x B
  b          : 1 x 1 x 2 x B
  wavelength : 33

Outputs:
  Sr,Sg,Sb   : 1 x 1 x 33 x B
  nbatch = size(b,2);
"""



# PCA model
S = np.dot(PC, b) + mu  # Matrix multiplication + adding mean vector

# ReLU activation (clamp negative values to zero)
S = np.maximum(S, 0)

# Split S into Sr, Sg, Sb
Sr = np.reshape(S[:wavelength, :], (1, 1, wavelength, nbatch))
Sg = np.reshape(S[wavelength:wavelength*2, :], (1, 1, wavelength, nbatch))
Sb = np.reshape(S[wavelength*2:wavelength*3, :], (1, 1, wavelength, nbatch))

# Optionally assign a name if necessary
S_name = 'S'




In [ ]:
#computeSpecularities.m

def computeSpecularities(specmask,lightcolour):

"""
Inputs:
  specmask          : H x W x 1 x B
  lightcolour      : 1 x 1 x 3 x B
Output:
  Specularities    : H x W x 1 x B
"""
Specularities = specmask * lightcolour


In [ ]:
#computelightcolour.m

def computelightcolour(e,Sr,Sg,Sb):
  """
Inputs:
  Sr,Sg,Sb         : 1 x 1 x 33 x B
  e                : 1 x 1 x 33 x B
Output:
  lightcolour        : 1 x 1 x 3 x B

"""
------------------------ lightcolour ---------------------------------------------

# Element-wise multiplication and summation along the third axis (axis=2 in Python)
lightcolour = np.array([np.sum(Sr * e, axis=2), np.sum(Sg * e, axis=2), np.sum(Sb * e, axis=2)])

# Reshape to (1, 1, 3, size_of_last_dimension)
lightcolour = lightcolour.reshape((1, 1, 3, lightcolour.shape[1]))



In [ ]:
#fromRawTosRGB.m

def fromRawTosRGB(imWB,T_RAW2XYZ):

  """

Inputs:
 imWB: H X W X 3 X B
 Tmatrix :  128 X 128 X 9
Output:
 sRGBim : H x W x 3 x B

 """

  # Compute Ix, Iy, Iz
Ix = (T_RAW2XYZ[0, 0, 0, :] * imWB[:, :, 0, :] + T_RAW2XYZ[0, 0, 3, :] * imWB[:, :, 1, :] + T_RAW2XYZ[0, 0, 6, :] * imWB[:, :, 2, :])

Iy = (T_RAW2XYZ[0, 0, 1, :] * imWB[:, :, 0, :] + T_RAW2XYZ[0, 0, 4, :] * imWB[:, :, 1, :] + T_RAW2XYZ[0, 0, 7, :] * imWB[:, :, 2, :])

Iz = (T_RAW2XYZ[0, 0, 2, :] * imWB[:, :, 0, :] + T_RAW2XYZ[0, 0, 5, :] * imWB[:, :, 1, :] + T_RAW2XYZ[0, 0, 8, :] * imWB[:, :, 2, :])

# Concatenate Ix, Iy, Iz along the third dimension
Ixyz = np.stack((Ix, Iy, Iz), axis=2)

# Define the transformation matrix
Txyzrgb = np.array([[3.2406, -1.5372, -0.4986],[-0.9689,  1.8758,  0.0415], [0.0557, -0.2040,  1.057]], dtype=np.float32)

# Check if imWB is a special layer
if isinstance(imWB, Layer):
    Txyzrgb = Param(value=Txyzrgb, learning_rate=0)
    Txyzrgb.name = 'Txyzrgb'

# Compute R, G, B channels
R = (Txyzrgb[0, 0] * Ixyz[:, :, 0, :] + Txyzrgb[0, 1] * Ixyz[:, :, 1, :] + Txyzrgb[0, 2] * Ixyz[:, :, 2, :])

G = (Txyzrgb[1, 0] * Ixyz[:, :, 0, :] + Txyzrgb[1, 1] * Ixyz[:, :, 1, :] + Txyzrgb[1, 2] * Ixyz[:, :, 2, :])

B = (Txyzrgb[2, 0] * Ixyz[:, :, 0, :] + Txyzrgb[2, 1] * Ixyz[:, :, 1, :] + Txyzrgb[2, 2] * Ixyz[:, :, 2, :])

# Concatenate R, G, B along the third dimension
sRGBim = np.stack((R, G, B), axis=2)

# Apply ReLU activation (clamp negative values to zero)
sRGBim = np.maximum(sRGBim, 0)


In [ ]:
#BiotoSpectralRef.m

def BiotoSpectralRef(fmel,fblood,Newskincolour):

"""

Inputs:
  fmel             : H x W x 1 x B
  fblood           : H x W x 1 x B
  Newskincolour    : 256 x 256 x 33 x B
Output:
  R_total          : H x W x 33 x B

"""

import torch
import torch.nn.functional as F

# Concatenate fblood and fmel along the third dimension
BiophysicalMaps = torch.cat((fblood, fmel), dim=2)

# Permute the dimensions: 2 x H x W x B -> B x H x W x 2
BiophysicalMaps = BiophysicalMaps.permute(3, 1, 2, 0)


# Perform bilinear interpolation using PyTorch

R_total = F.interpolate(Newskincolour, size=BiophysicalMaps.shape[2:], mode='bilinear', align_corners=False)